In [49]:
import torch
print(torch.__version__)

2.5.1+cu124


In [4]:
torch.manual_seed(42)

In [4]:
x = torch.Tensor([1, 2, 3])
print(x)
print(type(x))

y = torch.rand(3)
print(y)

tensor([1., 2., 3.])
<class 'torch.Tensor'>
tensor([0.8823, 0.9150, 0.3829])


In [5]:
tensor = torch.arange(4)
x1 = torch.rand(2, 3)
x2 = torch.rand(2, 3)

print(x1)
print(x2)
y = x1 - x2
print(y)

tensor([[0.9593, 0.3904, 0.6009],
        [0.2566, 0.7936, 0.9408]])
tensor([[0.1332, 0.9346, 0.5936],
        [0.8694, 0.5677, 0.7411]])
tensor([[ 0.8261, -0.5441,  0.0073],
        [-0.6128,  0.2259,  0.1997]])


In [6]:
sd = torch.arange(5, 10, 1)
sd1 = torch.rand(2, 6, 4)
print(sd)
print(sd1.view(1,1, len(sd1.flatten())))

tensor([5, 6, 7, 8, 9])
tensor([[[0.4294, 0.8854, 0.5739, 0.2666, 0.6274, 0.2696, 0.4414, 0.2969,
          0.8317, 0.1053, 0.2695, 0.3588, 0.1994, 0.5472, 0.0062, 0.9516,
          0.0753, 0.8860, 0.5832, 0.3376, 0.8090, 0.5779, 0.9040, 0.5547,
          0.3423, 0.6343, 0.3644, 0.7104, 0.9464, 0.7890, 0.2814, 0.7886,
          0.5895, 0.7539, 0.1952, 0.0050, 0.3068, 0.1165, 0.9103, 0.6440,
          0.7071, 0.6581, 0.4913, 0.8913, 0.1447, 0.5315, 0.1587, 0.6542]]])


In [46]:
x = torch.ones(3)

print(x.requires_grad)
x.requires_grad_(True)
print(x.requires_grad)


False
True


In [47]:
x = torch.arange(3, dtype=torch.float32, requires_grad=True)
print("X: ", x)

X:  tensor([0., 1., 2.], requires_grad=True)


In [48]:
a = x + 2
b = a ** 2
c = b + 1
y = c.mean()
print("Y: ", y)

print(x)

Y:  tensor(10.6667, grad_fn=<MeanBackward0>)
tensor([0., 1., 2.], requires_grad=True)


In [49]:
y.backward()
print(x.grad)

tensor([1.3333, 2.0000, 2.6667])


In [5]:
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print (x)
else:
    print ("MPS device not found.")

MPS device not found.


In [6]:
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cuda")
print("Device", device)

Device cuda


In [7]:
print(torch.cuda.is_available())
print(torch.version.cuda)

True
12.4


In [8]:
!nvidia-smi

Thu Jan  2 16:45:58 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 566.36                 Driver Version: 566.36         CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1070      WDDM  |   00000000:01:00.0  On |                  N/A |
|  0%   37C    P8             14W /  230W |    1301MiB /   8192MiB |      1%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [9]:
x = torch.zeros(2, 3)
x = x.to(device)
print("X", x)

X tensor([[0., 0., 0.],
        [0., 0., 0.]], device='cuda:0')


In [10]:
import time
x = torch.randn(10000, 10000)

start_time = time.time()
_ = torch.matmul(x, x)
end_time = time.time()

print(f"CPU Time: {end_time - start_time}")

x = x.to(device)
_ = torch.matmul(x, x)

start_time = time.time()
_ = torch.matmul(x, x)
torch.mps.synchronize()
end_time = time.time()

print(f"MPS Time: {end_time - start_time}")


CPU Time: 4.699336290359497


RuntimeError: Cannot execute deviceSynchronize() without MPS backend.

In [ ]:
torch.manual_seed(1234)
TENSOR_A_CPU = torch.rand(100000, 100000)
TENSOR_B_CPU = torch.rand(100000, 100000)

torch.manual_seed(1234)
TENSOR_A_MPS = torch.rand(100000, 100000).to(device)
TENSOR_B_MPS = torch.rand(100000, 100000).to(device)

# Warm-up
for _ in range(100):
    torch.matmul(torch.rand(500,500).to(device), torch.rand(500,500).to(device))
    
start_time = time.time()
torch.matmul(TENSOR_A_CPU, TENSOR_B_CPU)
print("CPU : --- %s seconds ---" % (time.time() - start_time))

start_time = time.time()
torch.matmul(TENSOR_A_MPS, TENSOR_B_MPS)
print("MPS : --- %s seconds ---" % (time.time() - start_time))

CPU : --- 3.605742931365967 seconds ---
MPS : --- 0.0023162364959716797 seconds ---


In [21]:
import torch
import time

# Define a function to measure execution time
def benchmark(device, size=1000):
    # Create random tensors
    x = torch.randn(size, size, device=device)
    y = torch.randn(size, size, device=device)
    
    # Warm-up to ensure fair measurement
    for _ in range(10):
        _ = torch.mm(x, y)
        if device == 'cuda':
            torch.cuda.synchronize()
    
    # Measure execution time
    start_time = time.time()
    for _ in range(1000):  # Perform the operation multiple times
        _ = torch.mm(x, y)
        if device == 'cuda':
            torch.cuda.synchronize()  # Synchronize after each operation
    
    elapsed_time = time.time() - start_time
    return elapsed_time

# Test on CPU
cpu_time = benchmark('cpu')
print(f"CPU time: {cpu_time:.4f} seconds")

# Test on GPU if available
if torch.cuda.is_available():
    gpu_time = benchmark('cuda')
    print(f"GPU time: {gpu_time:.4f} seconds")
else:
    print("CUDA is not available on this system.")


CPU time: 6.4453 seconds
GPU time: 0.7341 seconds


In [22]:
import torch.nn as nn

import torch.nn.functional as F

In [23]:
class SimpleClassifier(nn.Module):
    def __init__(self, num_inputs, num_hidden, num_outputs):
        super().__init__()

        self.linear1 = nn.Linear(num_inputs, num_hidden)
        self.act_fn = nn.Tanh()
        self.linear2 = nn.Linear(num_hidden, num_outputs)

    def forward(self, x):
        x = self.linear1(x)
        x = self.act_fn(x)
        x = self.linear2(x)
        return x

In [24]:
model = SimpleClassifier(num_inputs=3, num_hidden=4, num_outputs=1)
print(model)

SimpleClassifier(
  (linear1): Linear(in_features=3, out_features=4, bias=True)
  (act_fn): Tanh()
  (linear2): Linear(in_features=4, out_features=1, bias=True)
)


In [25]:
for name, param in model.named_parameters():
    print(name, param.shape)

linear1.weight torch.Size([4, 3])
linear1.bias torch.Size([4])
linear2.weight torch.Size([1, 4])
linear2.bias torch.Size([1])


In [43]:
import torch.utils.data as data

class XORDataset(data.Dataset):
    def __init__(self, size, std=0.1):
        super().__init__()
        self.size = size
        self.std = std
        self.generate_continuous_xor()

    def generate_continuous_xor(self):
        data = torch.randint(low=0, high=2, size=(self.size, 2), dtype=torch.float32)
        label = (data.sum(dim=1) == 1).to(torch.long)
        data += self.std * torch.randn(data.shape)

        self.data = data
        self.label = label

    def __len__(self):
        return self.size

    def __getitem__(self, idx):
        data_point = self.data[idx]
        data_label = self.label[idx]
        return data_point, data_label

In [44]:
dataset = XORDataset(size=200)
print("Size of dataset: ", len(dataset))
print("Data point 0: ", dataset[0])

Size of dataset:  200
Data point 0:  (tensor([0.9319, 0.0590]), tensor(1))


In [48]:
loss_module = nn.BCEWithLogitsLoss()

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr = 0.1)

In [ ]:
def train_model(model: nn.Module, optimizer: nn.)